### Задание 2
#### Условие
VI. Г. И. Марчук предложил модель, которая описывает борьбу вирусов
$N_1 (t)$, антител $N_2 (t)$, и плазматических клеток $N_3 (t)$ в человеческом организме,
зараженном вирусной инфекцией. Мера зараженности определяется переменной
величиной $m(t)$. Модель Марчука описывается следующей системой уравнений:
\begin{cases}
 \dot N_1(t) = (\epsilon - \gamma N_2(t))N_1(t), \\
 \dot N_2(t) = \rho N_3(t) - (\mu + \nu \gamma N_1(t))N_2(t),(2) \\
 \dot N_3(t) = \alpha N_1(t) N_2(t) - N_3(t) (N_3 - N_3^*) \\
 \dot m(t) = \sigma N_1 - \mu m(t).
\end{cases}
Здесь $\epsilon = 2$, $\gamma = 0.8$, $\rho = 0.17$, $\mu = 0.5$, $\nu \gamma = 8$; $\alpha = 10^4$ , если $m ≤ 0.1$;
$\alpha = 10^5(1 − m)/9$, если $0.1 ≤ m ≤ 1$; $\sigma > 0$.
Задачи:
1. Найти положения равновесия системы (2).
2. Исследовать на устойчивость положения равновесия системы (2).
3. Построить фазовые портреты системы (2) для разных начальных условий.
4. Установить бифуркационные значения параметра $\sigma$ для $N_3(t)$.

#### Решение задачи 2

In [2]:
%matplotlib notebook
from numpy import *
import pylab as p
from scipy.integrate import odeint

# Definition of parameters

def dN_dt(N, t=0):
    return array([ (eps - gamma*N[1])*N[0],
                   rho * N[2] - (mu + nu*gamma*N[0])*N[1],
                  alpha*N[0]*N[1] - N[2]*(N[2] - N2_lag),
                  sigma*N[0] - mu*N[3]
                 ])

In [3]:
#parameters
eps = 2
gamma = 0.8
rho = 0.17
mu = 0.5
nu = 10
alpha = 10000
m = 0.09
# another paramerets
